In [1]:
dataFile = '/content/drive/MyDrive/combopreppedMafia.tsv'

In [ ]:
#https://huggingface.co/docs/transformers/model_doc/roberta

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb#scrollTo=1i6nzMG0vQi5

In [3]:
!pip install datasets
!pip install evaluate
!pip install transformers==4.48.0 #4.51.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [4]:
import transformers
import datasets
import random
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, RobertaForSequenceClassification
from datasets import Dataset
import evaluate



In [5]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", use_fast=True)
#labelIDS = {"T": 0, "D": 1}
def preprocess_function(examples):
    label_mapping = {"T": 0, "D": 1}

    res  = tokenizer(examples["text"], padding=True, truncation=True, max_length=512)
    res['labels'] = [label_mapping[label] for label in examples['labels']]

  #  res["labels"] = labelIDS[examples["labels"]]
  #  print(res)
    return res

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
import sklearn
from sklearn.model_selection import train_test_split

df=pd.read_csv(dataFile, sep='\t')
print(df)

df = df.drop(["game", "Unnamed: 0.1", "Unnamed: 0", "msgAndHistory", "message", "history","mId", "role","name", "time" , 'withName', 'Unnamed: 0.2', 'hiscombo'], axis=1)
#df['msgAndHistory'] = df.apply(dataFix, axis = 1)
df = df.rename(columns={'label': 'labels','nohiscombo': 'text'})
print(df)

train, validate = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)

print(df['text'][1])
mtrain = Dataset.from_pandas(train)
meval = Dataset.from_pandas(validate)

      Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  mId             name       role  \
0                0             0         431    2   Natalie Morris      mafia   
1                1             1         422    3      Ryan Hodges      mafia   
2                2             2         432    4   Natalie Morris      mafia   
3                3             3         423    5      Ryan Hodges      mafia   
4                4             4         413   10    Mary Trujillo  bystander   
...            ...           ...         ...  ...              ...        ...   
2202          2202          2202          53   98    Nicole Barker  bystander   
2203          2203          2203          67  101    Angela Murphy  bystander   
2204          2204          2204          68  102    Angela Murphy  bystander   
2205          2205          2205          69  104  Richard Stevens      mafia   
2206          2206          2206          70  105  Jessica Michael      mafia   

                           

In [7]:
d = df.loc[df["labels"] == "D"]
print(d)

     labels                                               text
0         D  {"Task" : "You will be given a message from th...
1         D  {"Task" : "You will be given a message from th...
2         D  {"Task" : "You will be given a message from th...
3         D  {"Task" : "You will be given a message from th...
7         D  {"Task" : "You will be given a message from th...
...     ...                                                ...
2187      D  {"Task" : "You will be given a message from th...
2192      D  {"Task" : "You will be given a message from th...
2198      D  {"Task" : "You will be given a message from th...
2205      D  {"Task" : "You will be given a message from th...
2206      D  {"Task" : "You will be given a message from th...

[813 rows x 2 columns]


In [8]:
#encoded_test = mtest.map(preprocess_function, batched=True)
encoded_eval = meval.map(preprocess_function, batched=True)
encoded_train = mtrain.map(preprocess_function, batched=True)


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

In [9]:
print(encoded_eval['labels'])

[1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]


In [10]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    pred = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=pred, references=labels)


In [11]:
from transformers import EarlyStoppingCallback


In [12]:
print(transformers.__version__)

4.48.0


In [13]:
#encoded_dataset = dataset.map(preprocess_function, batched=True)
labels = ["T", "D"]
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

#model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=2)
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels = 2, label2id=label2id, id2label=id2label)
batch_size = 16
args = TrainingArguments(
    "/content/drive/MyDrive/models/robertaMafianhCombofmt",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    #push_to_hub=True,
    report_to="none",


)


trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]

)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-f8fd204a2d94>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.737552,0.558559
2,No log,0.691596,0.558559
3,No log,0.686307,0.558559
4,0.671500,0.687720,0.558559
5,0.671500,0.686893,0.558559
6,0.671500,0.718031,0.558559


TrainOutput(global_step=786, training_loss=0.6693620754562262, metrics={'train_runtime': 391.9306, 'train_samples_per_second': 80.218, 'train_steps_per_second': 5.014, 'total_flos': 969399794592000.0, 'train_loss': 0.6693620754562262, 'epoch': 6.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.7375518083572388,
 'eval_accuracy': 0.5585585585585585,
 'eval_runtime': 0.9105,
 'eval_samples_per_second': 121.913,
 'eval_steps_per_second': 7.688,
 'epoch': 6.0}